In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_list, udf
from pyspark.sql.types import DoubleType

In [2]:
# Initialize Spark Session
spark = SparkSession.builder.appName("UDAFExample").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/24 14:06:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Sample Data
data = [("hello",), ("world",), ("!",), ("PySpark",)]
df = spark.createDataFrame(data, ["text"])

In [4]:
# Define the UDAF
def avg_length_udaf(values):
    if len(values) == 0:
        return 0.0
    total_length = sum(len(val) for val in values)
    return total_length / len(values)

In [5]:
# Register the UDAF
avg_length = udf(avg_length_udaf, DoubleType())

In [6]:
# Use the UDAF in a query
df_grouped = df.groupBy().agg(avg_length(collect_list("text")).alias("avg_length"))

In [7]:
# Show the result
df_grouped.show()

+----------+
|avg_length|
+----------+
|       4.5|
+----------+



In [8]:
# Stop the Spark session
spark.stop()